In [1]:
import re 
import numpy as np
import pandas as pd

In [2]:
file_path = './ranking-new.csv'
data = pd.read_csv(file_path)

In [5]:
data.head(5)

,排名,标题,评论数量,推荐率,出版社,作者,出版时间,价格
0,1,人生海海（麦家重磅力作，莫言、董卿盛赞，连续两年高居畅销榜，...,894784,100%,北京十月文艺出版社,麦家 著 ，新经典 出品,2019/4/16,¥55.00
1,2,大话中国艺术史（句句有梗，一口气读完10000年中国艺术史。艺术头...,41862,99.90%,海南出版社,意公子；读客文化 出品,2022/3/1,¥112.10
2,3,生死疲劳（不看不知道，莫言真幽默！全新版本！印章版和普通版随...,209949,100%,浙江文艺出版社,莫言；读客文化 出品,2022/1/1,¥66.40
3,4,蛤蟆先生去看心理医生（年销200万册！英国经典心理咨询入门书，知...,1077140,100%,天津人民出版社,罗伯特·戴博德，果麦文化 出品,2020/7/1,¥28.50
4,5,被讨厌的勇气：“自我启发之父”阿德勒的哲学课 岸见一郎,603965,100%,机械工业出版社,"岸見一郎, 古賀史健",2020/3/5,¥30.00


> 标题的词云

In [3]:
import jieba 
from wordcloud import WordCloud

In [13]:
TITLE_FILE_NAME = './titles.txt'

fp_title = open(TITLE_FILE_NAME, 'w+' , encoding='utf-8')
titles = data['标题']
for title in titles:
  fp_title.write(title[title.find('（'):] if title.find('（') != -1 else ' ')

# 把标题写入文件
fp_title.close()

# 修正标题
data['标题'].apply(lambda x: x[:x.find('（')] if x.find('（') != -1 else x)

0                             人生海海
1                          大话中国艺术史
2                             生死疲劳
3                       蛤蟆先生去看心理医生
4      被讨厌的勇气：“自我启发之父”阿德勒的哲学课 岸见一郎
                  ...             
495                        银河帝国：基地
496                  宫西达也“你肯定能行”绘本
497              天星童书·全球精选绘本：我的小影子
498                    哈利波特20周年纪念版
499                      森林100层的房子
Name: 标题, Length: 500, dtype: object

In [5]:

#注意到获取的页面内容中有html的标签和其他符号存在，不利于下一步的词云操作，所以需要利用jieba和正则表达式对得到的文本进行清洗
r ='[，。\%、；1234567890n-】【“”]《》（）'

file=open(TITLE_FILE_NAME, "r", encoding='utf-8').read()

# 剔除无关信息
file =re.sub(r,'',file)      
# 删除中文无关词汇
chinese_char = '[的版]'
file = re.sub(chinese_char,'',file)

#分词
con = jieba.lcut(file)     

#分词后插入空格
words = " ".join(con)

#词云分析
wordcloud = WordCloud(font_path="./assets/1570766042.ttf",background_color="white",width=1300, height=800).generate(words)

wordcloud.to_file('title_cloud.png') 

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.035 seconds.
Prefix dict has been built successfully.




从词云图中可以看出消费者更加倾向于购买标题中含有“全册”、“推荐”、“代表作”等词的书籍


> 评论数量

In [7]:
# 排序后的DataFrame
data_sort_by_comment = data.sort_values('评论数量', ascending=False)

# 评论数量最多的 top10、30
# data_sort_by_comment.head(10)
# data_sort_by_comment.head(30)

> top10出版社以及上榜的书籍

In [95]:
data_groupby_publisher = data.groupby(['出版社'])

# 将 DataFrameGroupBy 转换为 array 类型
# publisher_with_rank = np.array([list(i) for i in list(data_groupby_publisher)], dtype=object)

# 将 DataFrameGroupBy 转换为 array 类型
# 然后再每一个group（出版社）中添加这个出版社一共出版的图书
# 然后把这个array按照出版图书的降序排序
publisher_with_rank = np.array([np.insert(i, 0, i[1].shape[0]) for i in np.array([list(i) for i in list(data_groupby_publisher)], dtype=object)])
publisher_with_rank = publisher_with_rank[publisher_with_rank[:,0].argsort()][::-1]

# 所有出版社按照上榜图书排序和他们的图书
# publisher_with_rank

> 出版图书的定价，大家最喜欢什么样的价位的图书

> 出版年份和时间排序，大家最喜欢什么时候出版的书